In [1]:
import time
import pandas as pd
import numpy as np
from scipy.stats import rankdata, spearmanr
from scipy.spatial.distance import pdist, squareform
np.random.seed(1)

In [2]:
def generate_feature_distance_ranking(data, method='Pearson'):
    '''
    This function generates ranking of distances/dissimilarities between features for tabular data.

    Input:
    data: input data, n_sample by n_feature
    method: 'Pearson' uses Pearson correlation coefficient to evaluate similarity between features;
        'Spearman' uses Spearman correlation coefficient to evaluate similarity between features;
        'set' uses Jaccard index to evaluate similarity between features that are binary variables.

    Return:
    ranking: symmetric ranking matrix based on dissimilarity
    corr: matrix of distances between features
    '''

    num = data.shape[1]
    if method == 'Pearson':
        corr = np.corrcoef(np.transpose(data))
    elif method == 'Spearman':
        corr = spearmanr(data).correlation
    elif method == 'Euclidean':
        corr = squareform(pdist(np.transpose(data), metric='euclidean'))
        corr = np.max(corr) - corr
        corr = corr / np.max(corr)
    # This is the new set operation to calculate similarity. It does not tolerate all-zero features.
    elif method == 'set':
        corr1 = np.dot(np.transpose(data), data)
        corr2 = data.shape[0] - np.dot(np.transpose(1 - data), 1 - data)
        corr = corr1 / corr2

    corr = 1 - corr
    corr = np.around(a=corr, decimals=10)

    tril_id = np.tril_indices(num, k=-1)
    rank = rankdata(corr[tril_id])
    ranking = np.zeros((num, num))
    ranking[tril_id] = rank
    ranking = ranking + np.transpose(ranking)

    return ranking, corr

def generate_feature_distance_ranking_enhanced(data, method='Pearson'):
    '''
    This function generates ranking of distances/dissimilarities between features for tabular data.

    Input:
    data: input data, n_sample by n_feature
    method: 'Pearson' uses Pearson correlation coefficient to evaluate similarity between features;
        'Spearman' uses Spearman correlation coefficient to evaluate similarity between features;
        'set' uses Jaccard index to evaluate similarity between features that are binary variables.

    Return:
    ranking: symmetric ranking matrix based on dissimilarity
    corr: matrix of distances between features
    '''

    num = data.shape[1]
    if method == 'Pearson':
        corr = np.corrcoef(np.transpose(data))
    elif method == 'Spearman':
        corr = spearmanr(data).correlation
    elif method == 'Euclidean':
        corr = squareform(pdist(np.transpose(data), metric='euclidean'))
        corr = np.max(corr) - corr
        corr = corr / np.max(corr)
    # This is the new set operation to calculate similarity. It does not tolerate all-zero features.
    elif method == 'set':
        corr1 = np.dot(np.transpose(data), data)
        corr2 = data.shape[0] - np.dot(np.transpose(1 - data), 1 - data)
        corr = corr1 / corr2

    #corr = 1 - corr
    corr = np.around(a=corr, decimals=10)

    tril_id = np.tril_indices(num, k=-1)
    rank = rankdata(corr[tril_id])
    ranking = np.zeros((num, num))
    ranking[tril_id] = rank
    ranking = ranking + np.transpose(ranking)

    return ranking, corr

def generate_matrix_distance_ranking(num_r, num_c, method='Euclidean'):
    '''
    This function calculates the ranking of distances between all pairs of entries in a matrix of size num_r by num_c.

    Input:
    num_r: number of rows in the matrix
    num_c: number of columns in the matrix
    method: method used to calculate distance. Can be 'Euclidean' or 'Manhattan'.

    Return:
    coordinate: num_r * num_c by 2 matrix giving the coordinates of elements in the matrix.
    ranking: a num_r * num_c by num_r * num_c matrix giving the ranking of pair-wise distance.

    '''

    # generate the coordinates of elements in a matrix
    for r in range(num_r):
        if r == 0:
            coordinate = np.transpose(
                np.vstack((np.zeros(num_c), range(num_c))))
        else:
            coordinate = np.vstack((coordinate, np.transpose(
                np.vstack((np.ones(num_c) * r, range(num_c))))))

    # calculate the closeness of the elements
    num = num_r * num_c
    cord_dist = np.zeros((num, num))
    if method == 'Euclidean':
        for i in range(num):
            cord_dist[i, :] = np.sqrt(np.square(coordinate[i, 0] * np.ones(num) - coordinate[:, 0]) +
                                      np.square(coordinate[i, 1] * np.ones(num) - coordinate[:, 1]))
    elif method == 'Manhattan':
        for i in range(num):
            cord_dist[i, :] = np.abs(coordinate[i, 0] * np.ones(num) - coordinate[:, 0]) + \
                np.abs(coordinate[i, 1] * np.ones(num) - coordinate[:, 1])

    # generate the ranking based on distance
    tril_id = np.tril_indices(num, k=-1)
    rank = rankdata(cord_dist[tril_id])
    ranking = np.zeros((num, num))
    ranking[tril_id] = rank
    ranking = ranking + np.transpose(ranking)

    coordinate = np.int64(coordinate)
    return (coordinate[:, 0], coordinate[:, 1]), ranking


def IGTD_absolute_error(source, target, max_step=1000, switch_t=0, val_step=50, min_gain=0.00001, random_state=1,
                        save_folder=None, file_name=''):
    '''
    This function switches the order of rows (columns) in the source ranking matrix to make it similar to the target
    ranking matrix. In each step, the algorithm randomly picks a row that has not been switched with others for
    the longest time and checks all possible switch of this row, and selects the switch that reduces the
    dissimilarity most. Dissimilarity (i.e. the error) is the summation of absolute difference of
    lower triangular elements between the rearranged source ranking matrix and the target ranking matrix.

    Input:
    source: a symmetric ranking matrix with zero diagonal elements.
    target: a symmetric ranking matrix with zero diagonal elements. 'source' and 'target' should have the same size.
    max_step: the maximum steps that the algorithm should run if never converges.
    switch_t: the threshold to determine whether switch should happen
    val_step: number of steps for checking gain on the objective function to determine convergence
    min_gain: if the objective function is not improved more than 'min_gain' in 'val_step' steps,
        the algorithm terminates.
    random_state: for setting random seed.
    save_folder: a path to save the picture of source ranking matrix in the optimization process.
    file_name: a string as part of the file names for saving results

    Return:
    index_record: indices to rearrange the rows(columns) in source obtained the optimization process
    err_record: error obtained in the optimization process
    run_time: the time at which each step is completed in the optimization process
    '''

    np.random.RandomState(seed=random_state)
    if save_folder is not None:
        if os.path.exists(save_folder):
            shutil.rmtree(save_folder)
        os.mkdir(save_folder)

    source = source.copy()
    num = source.shape[0]
    tril_id = np.tril_indices(num, k=-1)
    index = np.array(range(num))
    index_record = np.empty((max_step + 1, num))
    index_record.fill(np.nan)
    index_record[0, :] = index.copy()

    # calculate the error associated with each row
    err_v = np.empty(num)
    err_v.fill(np.nan)
    for i in range(num):
        err_v[i] = np.sum(np.abs(source[i, 0:i] - target[i, 0:i])) + \
            np.sum(np.abs(source[(i + 1):, i] - target[(i + 1):, i]))

    step_record = -np.ones(num)
    err_record = [np.sum(abs(source[tril_id] - target[tril_id]))]
    pre_err = err_record[0]
    t1 = time.time()
    run_time = [0]

    for s in range(max_step):
        delta = np.ones(num) * np.inf

        # randomly pick a row that has not been considered for the longest time
        idr = np.where(step_record == np.min(step_record))[0]
        ii = idr[np.random.permutation(len(idr))[0]]

        for jj in range(num):
            if jj == ii:
                continue

            if ii < jj:
                i = ii
                j = jj
            else:
                i = jj
                j = ii

            err_ori = err_v[i] + err_v[j] - np.abs(source[j, i] - target[j, i])

            err_i = np.sum(np.abs(source[j, :i] - target[i, :i])) + \
                np.sum(np.abs(source[(i + 1):j, j] - target[(i + 1):j, i])) + \
                np.sum(np.abs(source[(j + 1):, j] - target[(j + 1):, i])
                       ) + np.abs(source[i, j] - target[j, i])
            err_j = np.sum(np.abs(source[i, :i] - target[j, :i])) + \
                np.sum(np.abs(source[i, (i + 1):j] - target[j, (i + 1):j])) + \
                np.sum(np.abs(source[(j + 1):, i] - target[(j + 1):, j])
                       ) + np.abs(source[i, j] - target[j, i])
            err_test = err_i + err_j - np.abs(source[i, j] - target[j, i])

            delta[jj] = err_test - err_ori

        delta_norm = delta / pre_err
        id = np.where(delta_norm <= switch_t)[0]
        if len(id) > 0:
            jj = np.argmin(delta)

            # Update the error associated with each row
            if ii < jj:
                i = ii
                j = jj
            else:
                i = jj
                j = ii
            for k in range(num):
                if k < i:
                    err_v[k] = err_v[k] - np.abs(source[i, k] - target[i, k]) - np.abs(source[j, k] - target[j, k]) + \
                        np.abs(source[j, k] - target[i, k]) + \
                        np.abs(source[i, k] - target[j, k])
                elif k == i:
                    err_v[k] = np.sum(np.abs(source[j, :i] - target[i, :i])) + \
                        np.sum(np.abs(source[(i + 1):j, j] - target[(i + 1):j, i])) + \
                        np.sum(np.abs(
                            source[(j + 1):, j] - target[(j + 1):, i])) + np.abs(source[i, j] - target[j, i])
                elif k < j:
                    err_v[k] = err_v[k] - np.abs(source[k, i] - target[k, i]) - np.abs(source[j, k] - target[j, k]) + \
                        np.abs(source[k, j] - target[k, i]) + \
                        np.abs(source[i, k] - target[j, k])
                elif k == j:
                    err_v[k] = np.sum(np.abs(source[i, :i] - target[j, :i])) + \
                        np.sum(np.abs(source[i, (i + 1):j] - target[j, (i + 1):j])) + \
                        np.sum(np.abs(
                            source[(j + 1):, i] - target[(j + 1):, j])) + np.abs(source[i, j] - target[j, i])
                else:
                    err_v[k] = err_v[k] - np.abs(source[k, i] - target[k, i]) - np.abs(source[k, j] - target[k, j]) + \
                        np.abs(source[k, j] - target[k, i]) + \
                        np.abs(source[k, i] - target[k, j])

            # switch rows i and j
            ii_v = source[ii, :].copy()
            jj_v = source[jj, :].copy()
            source[ii, :] = jj_v
            source[jj, :] = ii_v
            ii_v = source[:, ii].copy()
            jj_v = source[:, jj].copy()
            source[:, ii] = jj_v
            source[:, jj] = ii_v
            err = delta[jj] + pre_err

            # update rearrange index
            t = index[ii]
            index[ii] = index[jj]
            index[jj] = t

            # update step record
            step_record[ii] = s
            step_record[jj] = s
        else:
            # error is not changed due to no switch
            err = pre_err

            # update step record
            step_record[ii] = s

        err_record.append(err)
        #print('Step ' + str(s) + ' err: ' + str(err))
        index_record[s + 1, :] = index.copy()
        run_time.append(time.time() - t1)

        if s > val_step:
            if np.sum((err_record[-val_step - 1] - np.array(err_record[(-val_step):])) / err_record[
                    -val_step - 1] >= min_gain) == 0:
                break

        pre_err = err

    index_record = index_record[:len(err_record), :].astype(int)
    if save_folder is not None:
        pd.DataFrame(index_record).to_csv(save_folder + '/' + file_name + '_index.txt', header=False, index=False,
                                          sep='\t', line_terminator='\r\n')
        pd.DataFrame(np.transpose(np.vstack((err_record, np.array(range(s + 2))))),
                     columns=['error', 'steps']).to_csv(save_folder + '/' + file_name + '_error_and_step.txt',
                                                        header=True, index=False, sep='\t', line_terminator='\r\n')
        pd.DataFrame(np.transpose(np.vstack((err_record, run_time))), columns=['error', 'run_time']).to_csv(
            save_folder + '/' + file_name + '_error_and_time.txt', header=True, index=False, sep='\t',
            line_terminator='\r\n')

    return index_record, err_record, run_time


def IGTD_square_error(source, target, max_step=1000, switch_t=0, val_step=50, min_gain=0.00001, random_state=1,
                      save_folder=None, file_name=''):
    '''
    This function switches the order of rows (columns) in the source ranking matrix to make it similar to the target
    ranking matrix. In each step, the algorithm randomly picks a row that has not been switched with others for
    the longest time and checks all possible switch of this row, and selects the switch that reduces the
    dissimilarity most. Dissimilarity (i.e. the error) is the summation of squared difference of
    lower triangular elements between the rearranged source ranking matrix and the target ranking matrix.

    Input:
    source: a symmetric ranking matrix with zero diagonal elements.
    target: a symmetric ranking matrix with zero diagonal elements. 'source' and 'target' should have the same size.
    max_step: the maximum steps that the algorithm should run if never converges.
    switch_t: the threshold to determine whether switch should happen
    val_step: number of steps for checking gain on the objective function to determine convergence
    min_gain: if the objective function is not improved more than 'min_gain' in 'val_step' steps,
        the algorithm terminates.
    random_state: for setting random seed.
    save_folder: a path to save the picture of source ranking matrix in the optimization process.
    file_name: a string as part of the file names for saving results

    Return:
    index_record: ordering index to rearrange the rows(columns) in 'source' in the optimization process
    err_record: the error history in the optimization process
    run_time: the time at which each step is finished in the optimization process
    '''

    np.random.RandomState(seed=random_state)
    if save_folder is not None:
        if os.path.exists(save_folder):
            shutil.rmtree(save_folder)
        os.mkdir(save_folder)

    source = source.copy()
    num = source.shape[0]
    tril_id = np.tril_indices(num, k=-1)
    index = np.array(range(num))
    index_record = np.empty((max_step + 1, num))
    index_record.fill(np.nan)
    index_record[0, :] = index.copy()

    # calculate the error associated with each row
    err_v = np.empty(num)
    err_v.fill(np.nan)
    for i in range(num):
        err_v[i] = np.sum(np.square(source[i, 0:i] - target[i, 0:i])) + \
            np.sum(np.square(source[(i + 1):, i] - target[(i + 1):, i]))

    step_record = -np.ones(num)
    err_record = [np.sum(np.square(source[tril_id] - target[tril_id]))]
    pre_err = err_record[0]
    t1 = time.time()
    run_time = [0]

    for s in range(max_step):
        delta = np.ones(num) * np.inf

        # randomly pick a row that has not been considered for the longest time
        idr = np.where(step_record == np.min(step_record))[0]
        ii = idr[np.random.permutation(len(idr))[0]]

        for jj in range(num):
            if jj == ii:
                continue

            if ii < jj:
                i = ii
                j = jj
            else:
                i = jj
                j = ii

            err_ori = err_v[i] + err_v[j] - \
                np.square(source[j, i] - target[j, i])

            err_i = np.sum(np.square(source[j, :i] - target[i, :i])) + \
                np.sum(np.square(source[(i + 1):j, j] - target[(i + 1):j, i])) + \
                np.sum(np.square(source[(j + 1):, j] - target[(j + 1):, i])
                       ) + np.square(source[i, j] - target[j, i])
            err_j = np.sum(np.square(source[i, :i] - target[j, :i])) + \
                np.sum(np.square(source[i, (i + 1):j] - target[j, (i + 1):j])) + \
                np.sum(np.square(source[(j + 1):, i] - target[(j + 1):, j])
                       ) + np.square(source[i, j] - target[j, i])
            err_test = err_i + err_j - np.square(source[i, j] - target[j, i])

            delta[jj] = err_test - err_ori

        delta_norm = delta / pre_err
        id = np.where(delta_norm <= switch_t)[0]
        if len(id) > 0:
            jj = np.argmin(delta)

            # Update the error associated with each row
            if ii < jj:
                i = ii
                j = jj
            else:
                i = jj
                j = ii
            for k in range(num):
                if k < i:
                    err_v[k] = err_v[k] - np.square(source[i, k] - target[i, k]) - np.square(source[j, k] - target[j, k]) + \
                        np.square(source[j, k] - target[i, k]) + \
                        np.square(source[i, k] - target[j, k])
                elif k == i:
                    err_v[k] = np.sum(np.square(source[j, :i] - target[i, :i])) + \
                        np.sum(np.square(source[(i + 1):j, j] - target[(i + 1):j, i])) + \
                        np.sum(np.square(
                            source[(j + 1):, j] - target[(j + 1):, i])) + np.square(source[i, j] - target[j, i])
                elif k < j:
                    err_v[k] = err_v[k] - np.square(source[k, i] - target[k, i]) - np.square(source[j, k] - target[j, k]) + \
                        np.square(source[k, j] - target[k, i]) + \
                        np.square(source[i, k] - target[j, k])
                elif k == j:
                    err_v[k] = np.sum(np.square(source[i, :i] - target[j, :i])) + \
                        np.sum(np.square(source[i, (i + 1):j] - target[j, (i + 1):j])) + \
                        np.sum(np.square(
                            source[(j + 1):, i] - target[(j + 1):, j])) + np.square(source[i, j] - target[j, i])
                else:
                    err_v[k] = err_v[k] - np.square(source[k, i] - target[k, i]) - np.square(source[k, j] - target[k, j]) + \
                        np.square(source[k, j] - target[k, i]) + \
                        np.square(source[k, i] - target[k, j])

            # switch rows i and j
            ii_v = source[ii, :].copy()
            jj_v = source[jj, :].copy()
            source[ii, :] = jj_v
            source[jj, :] = ii_v
            ii_v = source[:, ii].copy()
            jj_v = source[:, jj].copy()
            source[:, ii] = jj_v
            source[:, jj] = ii_v
            err = delta[jj] + pre_err

            # update rearrange index
            t = index[ii]
            index[ii] = index[jj]
            index[jj] = t

            # update step record
            step_record[ii] = s
            step_record[jj] = s
        else:
            # error is not changed due to no switch
            err = pre_err

            # update step record
            step_record[ii] = s

        err_record.append(err)
        #print('Step ' + str(s) + ' err: ' + str(err))
        index_record[s + 1, :] = index.copy()
        run_time.append(time.time() - t1)

        if s > val_step:
            if np.sum((err_record[-val_step - 1] - np.array(err_record[(-val_step):])) / err_record[
                    -val_step - 1] >= min_gain) == 0:
                break

        pre_err = err

    index_record = index_record[:len(err_record), :].astype(int)
    if save_folder is not None:
        pd.DataFrame(index_record).to_csv(save_folder + '/' + file_name + '_index.txt', header=False, index=False,
                                          sep='\t', line_terminator='\r\n')
        pd.DataFrame(np.transpose(np.vstack((err_record, np.array(range(s + 2))))),
                     columns=['error', 'steps']).to_csv(save_folder + '/' + file_name + '_error_and_step.txt',
                                                        header=True, index=False, sep='\t', line_terminator='\r\n')
        pd.DataFrame(np.transpose(np.vstack((err_record, run_time))), columns=['error', 'run_time']).to_csv(
            save_folder + '/' + file_name + '_error_and_time.txt', header=True, index=False, sep='\t',
            line_terminator='\r\n')

    return index_record, err_record, run_time


def IGTD(source, target, err_measure='abs', max_step=1000, switch_t=0, val_step=50, min_gain=0.00001, random_state=1,
         save_folder=None, file_name=''):
    '''
    This is just a wrapper function that wraps the two search functions using different error measures.
    '''

    if err_measure == 'abs':
        index_record, err_record, run_time = IGTD_absolute_error(source=source,
                                                                 target=target, max_step=max_step, switch_t=switch_t, val_step=val_step, min_gain=min_gain,
                                                                 random_state=random_state, save_folder=save_folder, file_name=file_name)
    if err_measure == 'squared':
        index_record, err_record, run_time = IGTD_square_error(source=source,
                                                               target=target, max_step=max_step, switch_t=switch_t, val_step=val_step, min_gain=min_gain,
                                                               random_state=random_state, save_folder=save_folder, file_name=file_name)

    return index_record, err_record, run_time

In [3]:
X = np.random.rand(8, 6)
X

array([[4.17022005e-01, 7.20324493e-01, 1.14374817e-04, 3.02332573e-01,
        1.46755891e-01, 9.23385948e-02],
       [1.86260211e-01, 3.45560727e-01, 3.96767474e-01, 5.38816734e-01,
        4.19194514e-01, 6.85219500e-01],
       [2.04452250e-01, 8.78117436e-01, 2.73875932e-02, 6.70467510e-01,
        4.17304802e-01, 5.58689828e-01],
       [1.40386939e-01, 1.98101489e-01, 8.00744569e-01, 9.68261576e-01,
        3.13424178e-01, 6.92322616e-01],
       [8.76389152e-01, 8.94606664e-01, 8.50442114e-02, 3.90547832e-02,
        1.69830420e-01, 8.78142503e-01],
       [9.83468338e-02, 4.21107625e-01, 9.57889530e-01, 5.33165285e-01,
        6.91877114e-01, 3.15515631e-01],
       [6.86500928e-01, 8.34625672e-01, 1.82882773e-02, 7.50144315e-01,
        9.88861089e-01, 7.48165654e-01],
       [2.80443992e-01, 7.89279328e-01, 1.03226007e-01, 4.47893526e-01,
        9.08595503e-01, 2.93614148e-01]])

In [4]:
generate_feature_distance_ranking(X)

(array([[ 0.,  1., 14., 13., 11.,  2.],
        [ 1.,  0., 15., 12.,  5.,  8.],
        [14., 15.,  0.,  3.,  7.,  9.],
        [13., 12.,  3.,  0.,  4.,  6.],
        [11.,  5.,  7.,  4.,  0., 10.],
        [ 2.,  8.,  9.,  6., 10.,  0.]]),
 array([[0.        , 0.33489146, 1.61278693, 1.55405467, 1.07250942,
         0.56144746],
        [0.33489146, 0.        , 1.85490468, 1.51284323, 0.85546831,
         1.00558514],
        [1.61278693, 1.85490468, 0.        , 0.58737637, 0.99257009,
         1.0153641 ],
        [1.55405467, 1.51284323, 0.58737637, 0.        , 0.64979859,
         0.86573863],
        [1.07250942, 0.85546831, 0.99257009, 0.64979859, 0.        ,
         1.06034672],
        [0.56144746, 1.00558514, 1.0153641 , 0.86573863, 1.06034672,
         0.        ]]))

In [5]:
generate_feature_distance_ranking_enhanced(X)

(array([[ 0., 15.,  2.,  3.,  5., 14.],
        [15.,  0.,  1.,  4., 11.,  8.],
        [ 2.,  1.,  0., 13.,  9.,  7.],
        [ 3.,  4., 13.,  0., 12., 10.],
        [ 5., 11.,  9., 12.,  0.,  6.],
        [14.,  8.,  7., 10.,  6.,  0.]]),
 array([[ 1.        ,  0.66510854, -0.61278693, -0.55405467, -0.07250942,
          0.43855254],
        [ 0.66510854,  1.        , -0.85490468, -0.51284323,  0.14453169,
         -0.00558514],
        [-0.61278693, -0.85490468,  1.        ,  0.41262363,  0.00742991,
         -0.0153641 ],
        [-0.55405467, -0.51284323,  0.41262363,  1.        ,  0.35020141,
          0.13426137],
        [-0.07250942,  0.14453169,  0.00742991,  0.35020141,  1.        ,
         -0.06034672],
        [ 0.43855254, -0.00558514, -0.0153641 ,  0.13426137, -0.06034672,
          1.        ]]))

In [6]:
generate_matrix_distance_ranking(3, 2)

((array([0, 0, 1, 1, 2, 2], dtype=int64),
  array([0, 1, 0, 1, 0, 1], dtype=int64)),
 array([[ 0. ,  4. ,  4. ,  9.5, 12.5, 14.5],
        [ 4. ,  0. ,  9.5,  4. , 14.5, 12.5],
        [ 4. ,  9.5,  0. ,  4. ,  4. ,  9.5],
        [ 9.5,  4. ,  4. ,  0. ,  9.5,  4. ],
        [12.5, 14.5,  4. ,  9.5,  0. ,  4. ],
        [14.5, 12.5,  9.5,  4. ,  4. ,  0. ]]))

In [7]:
feature_ranking, _ = generate_feature_distance_ranking(X)
feature_ranking_en, _ = generate_feature_distance_ranking_enhanced(X)
matrix_coordinate, matrix_dist_ranking = generate_matrix_distance_ranking(3, 2)

In [8]:
index_record, err_record, run_time = IGTD(source=feature_ranking, target=matrix_dist_ranking, err_measure='squared')
index_record, err_record

(array([[0, 1, 2, 3, 4, 5],
        [4, 1, 2, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3, 0, 5],
        [4, 2, 1, 3,

In [9]:
index_record_en, err_record_en, run_time_en = IGTD(source=feature_ranking_en, target=matrix_dist_ranking, err_measure='squared')
index_record_en, err_record_en

(array([[0, 1, 2, 3, 4, 5],
        [5, 1, 2, 3, 4, 0],
        [2, 1, 5, 3, 4, 0],
        [2, 1, 5, 3, 4, 0],
        [3, 1, 5, 2, 4, 0],
        [3, 1, 5, 2, 4, 0],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2, 4, 5],
        [3, 1, 0, 2,

In [10]:
X_IGTD_en = np.reshape(X[:,index_record_en[-1]], (-1, 3, 2))
X_IGTD = np.reshape(X[:,index_record[-1]], (-1, 3, 2))

X_IGTD_en, X_IGTD

(array([[[3.02332573e-01, 7.20324493e-01],
         [4.17022005e-01, 1.14374817e-04],
         [1.46755891e-01, 9.23385948e-02]],
 
        [[5.38816734e-01, 3.45560727e-01],
         [1.86260211e-01, 3.96767474e-01],
         [4.19194514e-01, 6.85219500e-01]],
 
        [[6.70467510e-01, 8.78117436e-01],
         [2.04452250e-01, 2.73875932e-02],
         [4.17304802e-01, 5.58689828e-01]],
 
        [[9.68261576e-01, 1.98101489e-01],
         [1.40386939e-01, 8.00744569e-01],
         [3.13424178e-01, 6.92322616e-01]],
 
        [[3.90547832e-02, 8.94606664e-01],
         [8.76389152e-01, 8.50442114e-02],
         [1.69830420e-01, 8.78142503e-01]],
 
        [[5.33165285e-01, 4.21107625e-01],
         [9.83468338e-02, 9.57889530e-01],
         [6.91877114e-01, 3.15515631e-01]],
 
        [[7.50144315e-01, 8.34625672e-01],
         [6.86500928e-01, 1.82882773e-02],
         [9.88861089e-01, 7.48165654e-01]],
 
        [[4.47893526e-01, 7.89279328e-01],
         [2.80443992e-01, 1.03226

In [16]:
len(err_record), len(err_record_en)

(53, 57)

In [12]:
np.savetxt('IGTD Illustration.csv', X, delimiter=',', fmt='%s')

In [13]:
np.savetxt('feature_rank.csv', feature_ranking, delimiter=',', fmt='%s')
np.savetxt('feature_rank_en.csv', feature_ranking_en, delimiter=',', fmt='%s')
np.savetxt('matrix_dist_rank.csv', matrix_dist_ranking, delimiter=',', fmt='%s')

In [14]:
err = []
for i in range(6):
    temp = feature_ranking.copy()
    temp[:, 0], temp[:, i] = temp[:, i], temp[:, 0]
    temp[0, :], temp[i, :] = temp[i, :], temp[0, :]
    err.append(np.sum((temp - matrix_dist_ranking) ** 2)/2)
err

[565.0, 527.0, 525.0, 585.0, 502.0, 555.0]

In [15]:
np.array(err) - err[0]

array([  0., -38., -40.,  20., -63., -10.])